### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Login: fabin99
Password: ········
Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
14,38366220,79085,10516,51530,0,0,944,2,2021-08-09 22:00,2021-08-09 22:00,0,2,2808,4,1
15,38553221,10516,79085,51530,0,0,944,3,2021-08-10 22:00,2021-08-10 22:00,1,4,2291,4,1
16,38360150,79085,89202,1001,2,62,944,4,2021-08-11 21:58,2021-08-11 21:58,5,0,2860,2,1
17,38560308,10516,79085,51530,0,0,944,5,2021-08-12 20:30,2021-08-12 20:30,3,0,3999,6,1
18,38560299,10516,79085,51530,0,0,944,6,2021-08-13 22:00,2021-08-13 22:00,5,1,4908,7,1
19,38560300,10516,79085,51530,0,0,945,0,2021-08-14 22:00,2021-08-14 22:00,6,0,3969,7,1
20,38426326,79085,121325,2074,1,54,945,1,2021-08-15 21:58,2021-08-15 21:59,3,0,3073,3,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

24

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,79085,41940,34534,2,2,2,0,0,5-2-3d,5,5,5,38366220
1,10516,28021,35427,1,12,0,0,0,alcinoo,5,6,10,38366220
0,10516,45747,35184,4,6,2,0,1,urubu,6,6,8,38553221
1,79085,24426,34989,2,18,0,0,0,5-2-3d,7,5,6,38553221
0,79085,0,0,0,0,0,0,0,5-2-3d,5,6,7,38360150
1,89202,0,0,0,0,0,0,0,4232,6,6,9,38360150
0,10516,29626,40289,0,12,3,2,0,LOIU,7,9,10,38560308
1,79085,40343,29680,1,3,3,1,0,5-2-3d,5,6,6,38560308
0,10516,30289,39182,2,18,1,0,0,KAMIKAZE OUTRO,8,9,10,38560299
1,79085,40287,31394,2,5,3,0,0,5-2-3d,6,5,6,38560299


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

48

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 2074

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,10467,1,3,1,0,0,5,0,3505005001004,2074
1,79085,1,3,1,0,0,3,0,3503003001005,2074
2,48360,1,3,1,0,0,3,0,3503003001002,2074
3,89684,1,3,1,0,0,1,0,3501001001008,2074
4,6335,1,0,0,0,1,0,1,499000000007,2074
5,121325,1,0,0,0,1,0,3,497000000006,2074
6,78774,1,0,0,0,1,0,3,497000000001,2074
7,121670,1,0,0,0,1,0,5,495000000003,2074


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

1

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()